Train on a random subset of the dcase dataset

*   507: len(train_subset)
*   110: len(valid_subset)


In [0]:
#pip install tensorflow==2.0.0

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import pandas as pd

print (tf.__version__)

2.2.0-rc4


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
#root_path = 'gdrive/My Drive/dissertation/dcase_dataset/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#LOADATA

#Get annotations  info txt
train_labels_path = "/content/gdrive/My Drive/dissertation/dcase_dataset/method1/train_labels[verified_1].csv"
valid_labels_path="/content/gdrive/My Drive/dissertation/dcase_dataset/method1/valid_labels[verified_1].csv"
train_labels=pd.read_csv(train_labels_path)
valid_labels=pd.read_csv(valid_labels_path)
print(type(train_labels))
print(train_labels.shape)

#get the corresponding rows     wavs
train_subset_path="/content/gdrive/My Drive/dissertation/dcase_dataset/subset/train"
valid_subset_path="/content/gdrive/My Drive/dissertation/dcase_dataset/subset/validation"
train_subset=os.listdir(train_subset_path)
valid_subset=os.listdir(valid_subset_path)
print(len(train_subset))

<class 'pandas.core.frame.DataFrame'>
(9690, 78)
507


In [0]:
#subset_labels
train_subset_labels=train_labels.loc[train_labels['audio_filename'].isin(train_subset)]
valid_subset_labels=valid_labels.loc[valid_labels['audio_filename'].isin(valid_subset)]

#reset_indexes
train_subset_labels.reset_index(inplace=True, drop=True, col_level='audio_filename')
valid_subset_labels.reset_index(inplace=True, drop=True, col_level='audio_filename')

print(train_subset_labels.shape)
print(valid_subset_labels.shape)

(422, 78)
(78, 78)


In [0]:
#rename files in dir (.wav.png -->.png) {DONT_RUN}
'''
stopwords = '.wav'
for filename in train_names:
  
  split=filename.split(".")
  resultafn  = [afn for afn in split if afn.lower() not in stopwords]
  result = '.'.join(resultafn)
  dir=str(train_features_path+'/')
  os.rename(dir+filename, dir+result)
  #print(dir)
  #print (result)

stopwords = '.wav'
for filename in valid_names:
  
  split=filename.split(".")
  resultafn  = [afn for afn in split if afn.lower() not in stopwords]
  result = '.'.join(resultafn)
  dir=str(valid_features_path+'/')
  os.rename(dir+filename, dir+result)
'''

In [0]:
#run once{subset_labels,features} #erase extension in order to compare names #rename the files
train_subset_labels['audio_filename'] = train_subset_labels['audio_filename'].str.split('.').str[0]
valid_subset_labels['audio_filename'] = valid_subset_labels['audio_filename'].str.split('.').str[0]

print(train_subset_labels['audio_filename'].head(5))
#print(valid_subset_labels['audio_filename'])

In [0]:
#get the corresponding feature names
train_names=train_subset_labels['audio_filename']
valid_names=valid_subset_labels['audio_filename']

train_names=train_names+'.png'
valid_names=valid_names+'.png'

'''
for i in range(len(train_names)):
  print('train',train_names[i])
  if(i<len(valid_names)):
    print('valid',valid_names[i])

print(type(train_names))
print(len(train_names))
print(len(valid_names))
'''

"\nfor i in range(len(train_names)):\n  print('train',train_names[i])\n  if(i<len(valid_names)):\n    print('valid',valid_names[i])\n\nprint(type(train_names))\nprint(len(train_names))\nprint(len(valid_names))\n"

In [0]:
#redeposit extension acronym accordingly 
train_subset_labels['audio_filename']=train_subset_labels['audio_filename']+'.wav'
valid_subset_labels['audio_filename']=valid_subset_labels['audio_filename']+'.wav'

'''
for i in range(len(train_subset_labels)):
  print(train_subset_labels['audio_filename'].iloc[i])
print(".........................")
for i in range(len(valid_subset_labels)):
  print(valid_subset_labels['audio_filename'].iloc[i])
'''

In [0]:
#Store features
train_features_path="/content/gdrive/My Drive/dissertation/dcase_dataset/features/subset/train"
valid_features_path="/content/gdrive/My Drive/dissertation/dcase_dataset/features/subset/validation"

train_fstack = [mpimg.imread(str(train_features_path+'/'+afn)) for afn in train_names]
train_features=np.stack(train_fstack, axis=0)

valid_fstack = [mpimg.imread(str(valid_features_path+'/'+afn)) for afn in valid_names]
valid_features=np.stack(valid_fstack, axis=0)

print(train_features.shape)
print(valid_features.shape)

(422, 288, 432, 4)
(78, 288, 432, 4)


In [0]:
#normalize values{Proved to be not neccessary}

#train_features=train_features /255.0
#valid_features=valid_features /255.0
#print(np.amax(train_features, axis=1))

print("TRAIN_FEATURES\n",train_features)
print("VALIDATION_FEATURES\n",valid_features)

In [0]:
#discard columns that arent needed
train_subset_labels=train_subset_labels.drop(['split','sensor_id','annotator_id', \
       '1-1_small-sounding-engine_proximity', \
       '1-2_medium-sounding-engine_proximity',  \
       '1-3_large-sounding-engine_proximity', \
       '1-X_engine-of-uncertain-size_proximity', '2-1_rock-drill_proximity',  \
       '2-2_jackhammer_proximity', '2-3_hoe-ram_proximity', \
       '2-4_pile-driver_proximity', \
       '2-X_other-unknown-impact-machinery_proximity',  \
       '3-1_non-machinery-impact_proximity', '4-1_chainsaw_proximity',  \
       '4-2_small-medium-rotating-saw_proximity', \
       '4-3_large-rotating-saw_proximity',  \
       '4-X_other-unknown-powered-saw_proximity', '5-1_car-horn_proximity', \
       '5-2_car-alarm_proximity', '5-3_siren_proximity',  \
       '5-4_reverse-beeper_proximity',  \
       '5-X_other-unknown-alert-signal_proximity',  \
       '6-1_stationary-music_proximity', '6-2_mobile-music_proximity',  \
       '6-3_ice-cream-truck_proximity', \
       '6-X_music-from-uncertain-source_proximity', \
       '7-1_person-or-small-group-talking_proximity', \
       '7-2_person-or-small-group-shouting_proximity',  \
       '7-3_large-crowd_proximity', '7-4_amplified-speech_proximity', \
       '7-X_other-unknown-human-voice_proximity', \
       '8-1_dog-barking-whining_proximity',],axis=1)

valid_subset_labels=valid_subset_labels.drop(['split','sensor_id','annotator_id', \
       '1-1_small-sounding-engine_proximity', \
       '1-2_medium-sounding-engine_proximity',  \
       '1-3_large-sounding-engine_proximity', \
       '1-X_engine-of-uncertain-size_proximity', '2-1_rock-drill_proximity',  \
       '2-2_jackhammer_proximity', '2-3_hoe-ram_proximity', \
       '2-4_pile-driver_proximity', \
       '2-X_other-unknown-impact-machinery_proximity',  \
       '3-1_non-machinery-impact_proximity', '4-1_chainsaw_proximity',  \
       '4-2_small-medium-rotating-saw_proximity', \
       '4-3_large-rotating-saw_proximity',  \
       '4-X_other-unknown-powered-saw_proximity', '5-1_car-horn_proximity', \
       '5-2_car-alarm_proximity', '5-3_siren_proximity',  \
       '5-4_reverse-beeper_proximity',  \
       '5-X_other-unknown-alert-signal_proximity',  \
       '6-1_stationary-music_proximity', '6-2_mobile-music_proximity',  \
       '6-3_ice-cream-truck_proximity', \
       '6-X_music-from-uncertain-source_proximity', \
       '7-1_person-or-small-group-talking_proximity', \
       '7-2_person-or-small-group-shouting_proximity',  \
       '7-3_large-crowd_proximity', '7-4_amplified-speech_proximity', \
       '7-X_other-unknown-human-voice_proximity', \
       '8-1_dog-barking-whining_proximity',],axis=1)

print(train_subset_labels.shape)
print(train_subset_labels.columns)

In [0]:
#GET_CLASS_NAMES
#Check MULTI INDEX HIERARCHICAL Taxonomy
class_names=train_subset_labels.columns
print(class_names)

In [0]:
#Visualize Feature Icons
#put it in a def...
#store features-normalize-build_model-train
show_range=10
for i in range(show_range):
  plt.figure(figsize=(4, 4))
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.title(train_names[i], loc='center')
  plt.imshow(train_features[i])
  print(train_subset_labels['audio_filename'].iloc[i])
  labels=[]
  for label in (class_names[9:]):#not for the spatiotemporal features
    if (train_labels[label].iloc[i]==1):
      labels.append(label)
  plt.xlabel(labels)
  plt.show()

In [0]:
train_subset_labels=train_subset_labels.drop(['audio_filename'], axis=1)
valid_subset_labels=valid_subset_labels.drop(['audio_filename'], axis=1)
                                             
print(train_subset_labels.columns)

In [0]:
print(len(train_subset_labels.columns))

45


In [0]:
#build_model

In [0]:
#train

In [0]:
print(train_features.shape)

(422, 288, 432, 4)


In [0]:
# Create a model
model =  models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation= 'relu', input_shape= (422,288,4)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation= 'relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(45, activation= 'softmax'))#(len(train_subset_labels.columns))==45

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 420, 286, 32)      1184      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 210, 143, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 208, 141, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 104, 70, 64)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 102, 68, 64)       36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 443904)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

In [0]:
#Define metrics
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

In [0]:
model.compile(optimizer= 'adam', 
              loss='sparse_categorical_crossentropy',
              metrics= METIRCS)
history= model.fit(train_images, train_labels, epochs=10,
                   validation_data=(test_images, test_labels))

In [0]:
#evaluation

plt.plot(history.history['accuracy'], label = ['accuracy'])
plt.plot(history.history['val_accuracy'], label = ['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc= 'lower right')

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose = 2)

In [0]:
print (test_acc)